In [1]:
!pip install -qq albumentations==1.0.3
!pip install wandb --upgrade
!pip install timm
# !conda install -y torchtext -c pytorch

In [2]:
import pandas as pd
import numpy as np
import os
import cv2
import timm
import torch
import torch.nn as nn
import albumentations as A
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torchmetrics
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold

In [3]:
print(f"PyTorch Lightning version: {pl.__version__}")

In [4]:
DEBUG = False

class CFG:
    proj_name = 'Sorghum Kaggle Competition'
    seed = 42
    model_name = 'tf_efficientnet_b3_ns'
    pretrained = True
    img_size = 512
    num_classes = 100
    lr = 1e-4
    max_lr = 1e-3
    pct_start = 0.2
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    num_epochs = 20
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 4
    penalty = 2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
seed_everything(CFG.seed)

In [6]:
PATH = "../input/sorghum-id-fgvc-9/"

# TRAIN_DIR = PATH + 'train_images/'
TRAIN_DIR = "../input/sorghum-id-fgvc-9/train_images/"
TEST_DIR = PATH + 'test/'

In [7]:
data = pd.read_csv(PATH + "train_cultivar_mapping.csv")
print(len(data))
data.dropna(inplace=True)
print(len(data))
data.head()

In [8]:
def remove_missing_images(data):
    images = data['image'].values
    indices = []
    for i in range(data.shape[0]):
        im = data.image.iloc[i]
        if not os.path.exists(os.path.join(TRAIN_DIR, im)):
            indices.append(i)
    data = data.drop(indices, axis=0).reset_index(drop=True)
    return data

data = remove_missing_images(data)

In [9]:
data.shape

In [10]:
unique_cultivars = list(data["cultivar"].unique())
num_classes = len(unique_cultivars)

CFG.num_classes = num_classes
print(num_classes)

In [11]:
data["file_path"] = data["image"].apply(lambda image: TRAIN_DIR + image)
data["cultivar_index"] = data["cultivar"].map(lambda item: unique_cultivars.index(item))

In [12]:
if DEBUG == True:
    data = data[:200]
    CFG.num_epochs = 10

In [13]:
skf = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)

for train_idx, valid_idx in skf.split(data['image'], data["cultivar_index"]):
    df_train = data.iloc[train_idx]
    df_valid = data.iloc[valid_idx]

print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")

print(df_train.cultivar.value_counts())
print(df_valid.cultivar.value_counts())

In [14]:
class SorghumDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_path = df['file_path'].values
        self.labels = df["cultivar_index"].values
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        image_path = self.image_path[idx]
        image = cv2.imread(image_path)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        return {'image':image, 'target': label}

In [15]:
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.Flip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ], p=0.5),
            A.OneOf([
                A.Blur(p=0.1),
                A.GaussianBlur(p=0.1),
                A.MotionBlur(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.GaussNoise(p=0.1),
                A.ISONoise(p=0.1),
                A.GridDropout(ratio=0.5, p=0.2),
                A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
            ], p=0.2),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [16]:
class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=CFG.penalty,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [17]:
train_dataset = SorghumDataset(df_train, get_transform('train'))
valid_dataset = SorghumDataset(df_valid, get_transform('valid'))

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, pin_memory=True, drop_last=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, pin_memory=True, num_workers=2)

In [18]:
CFG.steps_per_epoch = len(train_loader)
CFG.steps_per_epoch

In [19]:
class CustomEffNet(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, CFG.num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [20]:
class LitSorghum(pl.LightningModule):
    def __init__(self, model):
        super(LitSorghum, self).__init__()
        self.model = model
        self.metric = torchmetrics.Accuracy(threshold=0.5, num_classes=CFG.num_classes)
        self.criterion = FocalLoss()
        self.lr = CFG.lr

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, 
                                                             epochs=CFG.num_epochs, steps_per_epoch=CFG.steps_per_epoch,
                                                             max_lr=CFG.max_lr, pct_start=CFG.pct_start, 
                                                             div_factor=CFG.div_factor, final_div_factor=CFG.final_div_factor)
        scheduler = {'scheduler': self.scheduler, 'interval': 'step',}

        return [self.optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        image = batch['image']
        target = batch['target'].long()
        output = self.model(image)
        loss = self.criterion(output, target)
        score = self.metric(output.argmax(1), target)
        logs = {'train_loss': loss, 'train_acc': score, 'lr': self.optimizer.param_groups[0]['lr']}
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        image = batch['image']
        target = batch['target'].long()
        output = self.model(image)
        loss = self.criterion(output, target)
        score = self.metric(output.argmax(1), target)
        logs = {'valid_loss': loss, 'valid_acc': score}
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

In [21]:
model = CustomEffNet(model_name=CFG.model_name, pretrained=CFG.pretrained)

In [22]:
lit_model = LitSorghum(model.model)

In [23]:
logger = CSVLogger(save_dir='logs/', name=CFG.model_name)
logger.log_hyperparams(CFG.__dict__)
checkpoint_callback = ModelCheckpoint(monitor='valid_loss',
                                      save_top_k=1,
                                      save_last=True,
                                      save_weights_only=True,
                                      filename='{epoch:02d}-{valid_loss:.4f}-{valid_acc:.4f}',
                                      verbose=False,
                                      mode='min')

trainer = Trainer(
    max_epochs=CFG.num_epochs,
    gpus=[0],
    accumulate_grad_batches=CFG.accum,
    precision=CFG.precision,
    callbacks=[checkpoint_callback], 
    logger=logger,
    weights_summary='top',
)

In [24]:
trainer.fit(lit_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

In [25]:
!ls "./logs/tf_efficientnet_b3_ns/version_0/checkpoints/"

In [26]:
model = CustomEffNet(model_name=CFG.model_name, pretrained=False)
checkpoint = "./logs/tf_efficientnet_b3_ns/version_0/checkpoints/last.ckpt"
model.load_state_dict(torch.load(checkpoint)['state_dict'])

In [38]:
metrics = pd.read_csv(f'{trainer.logger.log_dir}/metrics.csv')
train_acc = metrics['train_acc'].dropna().reset_index(drop=True)
valid_acc = metrics['valid_acc'].dropna().reset_index(drop=True)

fig, axes = plt.subplots(1,3, figsize = (12,4))

axes[0].grid(True)
axes[0].plot(train_acc, color="r", marker="o", label='train/acc')
axes[0].plot(valid_acc, color="b", marker="x", label='valid/acc')
axes[0].legend(loc='lower right', fontsize=9)

train_loss = metrics['train_loss'].dropna().reset_index(drop=True)
valid_loss = metrics['valid_loss'].dropna().reset_index(drop=True)

axes[1].grid(True)
axes[1].plot(train_loss, color="r", marker="o", label='train/loss')
axes[1].plot(valid_loss, color="b", marker="x", label='valid/loss')
axes[1].legend(loc='upper right', fontsize=9)

lr = metrics['lr'].dropna().reset_index(drop=True)

axes[2].grid(True)
axes[2].plot(lr, color="g", marker="o", label='learning rate')
axes[2].legend(loc='upper right', fontsize=9)

In [28]:
sub = pd.read_csv(PATH + "sample_submission.csv")
sub.head()

In [1]:
sub = pd.read_csv(PATH + "sample_submission.csv")
sub["file_path"] = sub["filename"].apply(lambda image: TEST_DIR + image)
sub["cultivar_index"] = 0

test_dataset = SorghumDataset(sub, get_transform('valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

In [31]:
from tqdm import tqdm

model.cuda()
model.eval()

predictions = []
for batch in tqdm(test_loader):
    image = batch['image'].cuda()
    with torch.no_grad():
        outputs = model(image)
        preds = outputs.detach().cpu()
        predictions.append(preds.argmax(1))

In [32]:
tmp = predictions[0]
for i in range(len(predictions) - 1):
    tmp = torch.cat((tmp, predictions[i+1]))
    
predictions = [unique_cultivars[pred] for pred in tmp]

sub = pd.read_csv(PATH + "sample_submission.csv")
sub["cultivar"] = predictions
sub.to_csv('submission.csv', index=False)
sub.head()